In [9]:
import pandas as pd
from sklearn import preprocessing
from tensorflow.keras.layers.experimental import preprocessing as PP
import tensorflow as tf

import kerastuner as kt
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
train_data = pd.read_csv('..\\kaggle_data\\train.csv')

alpha = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]
alpha_conversion = {}
for index in range(0, len(alpha)):
    alpha_conversion[alpha[index]] = index / 29
alpha_conversion

train_data['cat0'].replace(alpha_conversion, inplace=True)
train_data['cat1'].replace(alpha_conversion, inplace=True)
train_data['cat2'].replace(alpha_conversion, inplace=True)
train_data['cat3'].replace(alpha_conversion, inplace=True)
train_data['cat4'].replace(alpha_conversion, inplace=True)
train_data['cat5'].replace(alpha_conversion, inplace=True)
train_data['cat6'].replace(alpha_conversion, inplace=True)
train_data['cat7'].replace(alpha_conversion, inplace=True)
train_data['cat8'].replace(alpha_conversion, inplace=True)
train_data['cat9'].replace(alpha_conversion, inplace=True)
train_data.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,0.000000,0.034483,0.0,0.000000,0.034483,0.103448,0.0,0.137931,0.068966,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,0.034483,0.000000,0.0,0.000000,0.034483,0.034483,0.0,0.137931,0.000000,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,0.000000,0.000000,0.0,0.068966,0.034483,0.103448,0.0,0.034483,0.068966,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,0.000000,0.000000,0.0,0.068966,0.034483,0.103448,0.0,0.137931,0.206897,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,0.000000,0.034483,0.0,0.000000,0.034483,0.034483,0.0,0.137931,0.068966,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [12]:
vector_data = train_data.drop(['id','target'],axis=1)
label_data = train_data[['target']]

training_vector_data = vector_data
training_label_data = label_data

In [13]:
def normalizedataframe(df):
    x = df.values  # returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df


normalizer = PP.Normalization()
normalizer.adapt(training_vector_data.to_numpy())

In [14]:
def root_mean_squared_error(y_true, y_pred):
    return tf.math.sqrt(tf.math.reduce_mean(tf.math.squared_difference(y_true, y_pred)))


def model_builder(hp):
    global normalizer

    this_model = tf.keras.Sequential()
    this_model.add(normalizer)

    hp_layer_one_units = hp.Int('layer_one_units', min_value=2, max_value=16, step=1)
    this_model.add(tf.keras.layers.Dense(units=hp_layer_one_units, activation='relu'))
    hp_layer_two_units = hp.Int('layer_two_units', min_value=2, max_value=16, step=1)
    this_model.add(tf.keras.layers.Dense(units=hp_layer_two_units, activation='relu'))

    this_model.add(tf.keras.layers.Dense(1))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    hp_momentum = hp.Int('momentum', min_value=1, max_value=9, step=1)

    tf.keras.optimizers.SGD(learning_rate=hp_learning_rate, momentum=hp_momentum / 10)
    this_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                       loss=root_mean_squared_error,
                       metrics=['mean_squared_error', 'accuracy'])

    return this_model

In [15]:
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=10,
                     factor=2,
                     directory='my_dir',
                     project_name='intro_to_kt9')

tuner.search(training_vector_data.values, training_label_data.values, epochs=50,
             validation_split=0.2)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=100)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('layer_one_units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 46 Complete [00h 04m 06s]
val_loss: 0.862825334072113

Best val_loss So Far: 0.859819769859314
Total elapsed time: 01h 10m 54s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 7 and the optimal learning rate for the optimizer
is 0.001.



In [16]:
model = tuner.hypermodel.build(best_hps)
model.fit(training_vector_data, training_label_data, epochs=10,
          validation_split=0.2)
model.save('Model2.h5')

Epoch 1/10
7500/7500 [==============================] - 25s 3ms/step - loss: 1.7728 - mean_squared_error: 6.0885 - accuracy: 0.0000e+00 - val_loss: 0.8684 - val_mean_squared_error: 0.7634 - val_accuracy: 0.0000e+00
Epoch 2/10
7500/7500 [==============================] - 24s 3ms/step - loss: 0.8711 - mean_squared_error: 0.7680 - accuracy: 0.0000e+00 - val_loss: 0.8685 - val_mean_squared_error: 0.7638 - val_accuracy: 0.0000e+00
Epoch 3/10
7500/7500 [==============================] - 24s 3ms/step - loss: 0.8690 - mean_squared_error: 0.7644 - accuracy: 0.0000e+00 - val_loss: 0.8659 - val_mean_squared_error: 0.7597 - val_accuracy: 0.0000e+00
Epoch 4/10
7500/7500 [==============================] - 24s 3ms/step - loss: 0.8663 - mean_squared_error: 0.7595 - accuracy: 0.0000e+00 - val_loss: 0.8652 - val_mean_squared_error: 0.7583 - val_accuracy: 0.0000e+00
Epoch 5/10
7500/7500 [==============================] - 24s 3ms/step - loss: 0.8668 - mean_squared_error: 0.7604 - accuracy: 0.0000e+00 - va

In [21]:
test = pd.read_csv('..\\kaggle_data\\test.csv')

alpha = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]
alpha_conversion = {}
for index in range(0, len(alpha)):
    alpha_conversion[alpha[index]] = index / 29
alpha_conversion

test['cat0'].replace(alpha_conversion, inplace=True)
test['cat1'].replace(alpha_conversion, inplace=True)
test['cat2'].replace(alpha_conversion, inplace=True)
test['cat3'].replace(alpha_conversion, inplace=True)
test['cat4'].replace(alpha_conversion, inplace=True)
test['cat5'].replace(alpha_conversion, inplace=True)
test['cat6'].replace(alpha_conversion, inplace=True)
test['cat7'].replace(alpha_conversion, inplace=True)
test['cat8'].replace(alpha_conversion, inplace=True)
test['cat9'].replace(alpha_conversion, inplace=True)
test.head()

vector_data = test.drop(['id'],axis=1)

sub = pd.read_csv('..\\kaggle_data\\sample_submission.csv')
predictions = model.predict(vector_data)

In [22]:
sub['target'] = predictions
sub.to_csv('submission.csv', index=False)